In [1]:
import requests
import os
import pathlib
from random import shuffle, seed
import numpy as np
import re
from subprocess import call
from time import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

In [3]:
vertical_spaces = '''
    <style>html *
        {font-size: 16px !important;
            line-height: 2.625 !important;
            color: #000000 !important;
            font-family: Nunito, sans-serif !important;}
    </style>
'''
hor_line_general ='''
<style>
    hr {
        display: block;
        margin-top: 0em;
        margin-bottom: 1.5em;
        margin-left: auto;
        margin-right: auto;
        border-style: inset;
        border-width: 1px;
        width:100%
    }
    </style>
    '''
for_language = vertical_spaces + '<head><meta charset="utf-8"><title>Tasks: </title></head>'

In [4]:
class Source_Scrapper:

    def __init__(self, url, task_num):
        self.url = url
        self.task_num = task_num
        self.first_step()

    def first_step(self):
        response = requests.get(self.url)

        soup = BeautifulSoup(response.content, 'html.parser')
        result_html = soup.prettify()
        with open('page_source.txt', 'w') as f:
            f.write(result_html)

        self.tasks = soup.find_all('replace_tag', {'class': 'replace_class_name'})
        self.answers = soup.find_all('replace_tag', {'class': 'replace_class_name'})


    def answers_extractor(self, answer):
        answer = str(answer)
        answer = answer.replace("replace_tag", 'S')
        answer = answer.replace(") );", "E")
        id = re.findall('"\d+"', answer)
        ans = re.findall("S'([^']*?)'E", answer)
        return [id, ans]
    
    def task_converter_teacher(self):
        assert len(self.tasks) == len(self.answers)

        tasks_final = []
        for idx, task in enumerate(self.tasks):
            task_answer = self.answers_extractor(self.answers[idx])
            task = str(task)
            task = task.replace(
                "replace_tag",
                f"Задание ({str(task_answer[0][0])[1:-1]})" + '\n',
            )
            task = task.replace("') );", 'END')
            idx_end = task.find("END")
            idx_start = task.find("Задание")
            soup = BeautifulSoup(task[idx_start + 7:idx_end], 'html.parser')
            pretty_task = soup.prettify()
            tasks_final.append(
                f'<br>{str(pretty_task)}<br>Answer: <br>{str(task_answer[1][0])}<hr>'
            )
        return tasks_final

    def task_converter_student(self):
        assert len(self.tasks) == len(self.answers)

        tasks_final = []
        for task in self.tasks:
            task = str(task)
            task = task.replace("replace_phrase", 'Задание ')
            task = task.replace("') );", 'END')
            idx_end = task.find("END")
            idx_start = task.find("Задание")
            soup = BeautifulSoup(task[idx_start + 7:idx_end], 'html.parser')
            pretty_task = soup.prettify()
            tasks_final.append(f'<br>{str(pretty_task)}<br><hr>')
        return tasks_final


    def write_to_file(self, student=True):
        if student:
            name = 'tutor_streamlit/streamlit_app/files/tasks/'
            tasks_final = self.task_converter_student()
        else:
            name = 'files/answers/'
            tasks_final = self.task_converter_teacher()
        seed(42)
        shuffle(tasks_final)
        for idx, elem in enumerate(tasks_final):
            mode = 'w' if idx == 0 else 'a'
            with open(f'{name}task_{str(self.task_num)}.html', mode) as f:
                if idx == 0:
                    f.write(f'{for_language}Задание ' + f"{idx + 1}. " + elem)
                else:
                    f.write(f"Задание {idx + 1}. {elem}")
        

In [18]:
urls = {
    2: 'link_2',
    3: 'link_3',
    5: 'link_5',
    6: 'link_6',
    7: 'link_7',
    8: 'link_8',
    9: 'link_9',
    11: 'link_11',
    12: 'link_12',
    13: 'link_13',
    14: 'link_14',
    15: 'link_15',
    16: 'link_16',
    17: 'link_17',
    18: 'link_18',
    19: 'link_19',
    22: 'link_22',
    23: 'link_23',
    24: 'link_24',
    25: 'link_25',
    26: 'link_26',
    27: 'link_27'
}

In [20]:
num = 22
obj = Source_Scrapper(urls[num], num)
obj.write_to_file(student=0)